In [1]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
from numpy import genfromtxt
import numpy as np
import csv

In [2]:
training_data = genfromtxt('training.csv',dtype = str, delimiter=',')

In [109]:
print(training_data[:2])

[['EventId' 'DER_mass_MMC' 'DER_mass_transverse_met_lep' 'DER_mass_vis'
  'DER_pt_h' 'DER_deltaeta_jet_jet' 'DER_mass_jet_jet'
  'DER_prodeta_jet_jet' 'DER_deltar_tau_lep' 'DER_pt_tot' 'DER_sum_pt'
  'DER_pt_ratio_lep_tau' 'DER_met_phi_centrality' 'DER_lep_eta_centrality'
  'PRI_tau_pt' 'PRI_tau_eta' 'PRI_tau_phi' 'PRI_lep_pt' 'PRI_lep_eta'
  'PRI_lep_phi' 'PRI_met' 'PRI_met_phi' 'PRI_met_sumet' 'PRI_jet_num'
  'PRI_jet_leading_pt' 'PRI_jet_leading_eta' 'PRI_jet_leading_phi'
  'PRI_jet_subleading_pt' 'PRI_jet_subleading_eta' 'PRI_jet_subleading_phi'
  'PRI_jet_all_pt' 'Weight' 'Label']
 ['100000' '138.47' '51.655' '97.827' '27.98' '0.91' '124.711' '2.666'
  '3.064' '41.928' '197.76' '1.582' '1.396' '0.2' '32.638' '1.017' '0.381'
  '51.626' '2.273' '-2.414' '16.824' '-0.277' '258.733' '2' '67.435' '2.15'
  '0.444' '46.062' '1.24' '-2.475' '113.497' '0.00265331133733' 's']]


In [123]:
train_labels = training_data[1:,-1]
train_labels = (train_labels == 's').astype(np.int32)
train_data = training_data[1:,1:31]
train_data = train_data.astype(np.float32)

In [124]:
a, b = np.where(train_data == -999)
new_r, new_c = np.where(train_data != -999)
mean_list = np.mean(train_data[new_r][new_c], axis = 0)

In [125]:
for i in range(len(a)):
    train_data[a[i]][train_data[a[i]] == -999] = mean_list[b[i]]
x, y = np.where(train_data == -999)
print(x,y)

(array([], dtype=int64), array([], dtype=int64))


In [129]:
print(Counter(train_data[:,22]))
data_jet_num_0 = train_data[train_data[:,22] == 0]
data_jet_num_0_labels = train_labels[np.where(train_data[:,22] == 0)]
print(Counter(data_jet_num_0_labels))
data_jet_num_1 = train_data[train_data[:,22] == 1]
data_jet_num_1_labels = train_labels[np.where(train_data[:,22] == 1)]
print(Counter(data_jet_num_1_labels))
data_jet_num_2 = train_data[train_data[:,22] == 2]
data_jet_num_2_labels = train_labels[np.where(train_data[:,22] == 2)]
print(Counter(data_jet_num_2_labels))
data_jet_num_3 = train_data[train_data[:,22] == 3]
data_jet_num_3_labels = train_labels[np.where(train_data[:,22] == 3)]
print(Counter(data_jet_num_3_labels))
data_jet_num_2_3 = np.concatenate((data_jet_num_2,data_jet_num_3))
data_jet_num_2_3_labels = np.concatenate((data_jet_num_2_labels,data_jet_num_3_labels))
print(Counter(data_jet_num_2_3_labels))
print(len(data_jet_num_0),len(data_jet_num_1),len(data_jet_num_2_3))

Counter({0.0: 99913, 1.0: 77544, 2.0: 50379, 3.0: 22164})
Counter({0: 74421, 1: 25492})
Counter({0: 49834, 1: 27710})
Counter({1: 25734, 0: 24645})
Counter({0: 15433, 1: 6731})
Counter({0: 40078, 1: 32465})
(99913, 77544, 72543)


In [120]:
data_jet_num_0 = np.delete(data_jet_num_0,[22],1)
print(data_jet_num_0.shape)
data_jet_num_1 = np.delete(data_jet_num_1,[22],1)
data_jet_num_2_3 = np.delete(data_jet_num_2_3,[22],1)

(99913, 29)


In [121]:
pca = PCA(0.99)
data_jet_num_0 = pca.fit_transform(data_jet_num_0)
print(pca.components_)
pca = PCA(0.99)
data_jet_num_1 = pca.fit_transform(data_jet_num_1)
print(pca.components_)
pca = PCA(0.99)
data_jet_num_2_3 = pca.fit_transform(data_jet_num_2_3)
print(pca.components_)

[[  2.74501909e-02   8.73614326e-02  -3.86257619e-02  -9.54644289e-03
    3.13440144e-01   3.13440174e-01   3.13440174e-01  -2.13815249e-03
   -9.54646524e-03  -1.19898422e-02   2.03224903e-04  -1.26303174e-03
    3.13440174e-01  -8.05597100e-03  -4.82847099e-05   1.11377944e-04
   -3.93389305e-03  -8.69047071e-05   9.72700855e-06   6.72260448e-02
   -6.68850407e-05  -5.23812957e-02   3.13440204e-01   3.13440204e-01
    3.13440174e-01   3.13440174e-01   3.13440174e-01   3.13440174e-01
    0.00000000e+00]
 [  5.56814075e-01   1.40702248e-01   4.74584371e-01   5.91806173e-02
    6.62114564e-03   6.62113819e-03   6.62113819e-03   3.41269351e-03
    5.91805913e-02   2.83239633e-01   3.78612283e-04   1.08266680e-03
    6.62116287e-03   1.38233587e-01   7.89787809e-05   4.15578761e-05
    1.45006031e-01   9.49531750e-05  -1.34107162e-04   8.05337727e-02
    9.80355071e-06   5.57341695e-01   6.62120525e-03   6.62120525e-03
    6.62116287e-03   6.62116287e-03   6.62116287e-03   6.62116287e-03


In [133]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, solver='lbfgs')
clf = clf.fit(data_jet_num_0[:60000], data_jet_num_0_labels[:60000])
print(clf.score(data_jet_num_0[60000:],data_jet_num_0_labels[60000:]))

clf = LogisticRegression(random_state=0, solver='lbfgs')
clf = clf.fit(data_jet_num_1[:50000], data_jet_num_1_labels[:50000])
print(clf.score(data_jet_num_1[50000:],data_jet_num_1_labels[50000:]))

clf = LogisticRegression(random_state=0, solver='lbfgs')
clf = clf.fit(data_jet_num_2_3[:50000], data_jet_num_2_3_labels[:50000])
print(clf.score(data_jet_num_2_3[50000:],data_jet_num_2_3_labels[50000:]))

0.811890862626
0.690567818763
0.706028478907


In [84]:
scaler = StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)

In [8]:
pca = PCA(.95)
pca.fit(train_data)
pca.n_components_
train_data = pca.transform(train_data)

In [9]:
print(pca.n_components_)
np.savetxt("training_data_pca.csv", train_data, delimiter=",")

15


In [10]:
# calculate Pearson's correlation
from scipy.stats import pearsonr
n = len(train_data[0])
dict1={}
for i in range(1,n-2):
    l= []
    for j in range(1,n-2):
        corr, _ = pearsonr(train_data[:,i], train_data[:,j])
        l.append(corr)
    dict1[training_data[0][i]]=l

dataframe = pd.DataFrame.from_dict(dict1, orient='index',columns=training_data[0][1:n-2])


#check for relations where there is 0.95 correlation
dict2={}
for key in dict1:
    n = len(dict1[key])
    for i in range(0,n):
        if dict1[key][i]>=0.95 and key!=training_data[0][i+1]:
            if key>training_data[0][i+1]:
                a = training_data[0][i+1]
                b = key
            else:
                a = key
                b = training_data[0][i+1]
            dict2[(a,b)]=dict1[key][i]

for key in dict2:
    print (key,dict2[key])

In [11]:
dataCsv = dataframe.to_csv(index=True)
f = open('correlation.csv','w+')
f.write(dataCsv)

In [12]:
# Plot of DER_mass_MC
import random
import numpy
from matplotlib import pyplot

filename='training.csv'
data = genfromtxt(filename,dtype = float, delimiter=',');
data1 = genfromtxt(filename,dtype = str, delimiter=',');
Mass = data[1:,1]
Y = data1[1:,32]
Y = np.array(Y)

Mass = np.array(Mass)
Msignal = Mass[Y=='s'];
Mbackground = Mass[Y=='b'];
bins = numpy.linspace(9, 500, 50)
pyplot.hist(Mbackground, bins, alpha=0.5, label='Background')
pyplot.hist(Msignal, bins, alpha=0.5, label='Signal')
pyplot.legend(loc='upper right')
pyplot.xlabel('DER_mass_MMC')
pyplot.ylabel('Number of events')
pyplot.show()

<Figure size 640x480 with 1 Axes>

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from collections import Counter

clf = GaussianNB()
clf = clf.fit(train_data[:200000], train_labels[:200000])
clf.score(train_data[200000:],train_labels[200000:])

0.72692000000000001

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
clf = clf.fit(train_data[:200000], train_labels[:200000])
clf.score(train_data[200000:],train_labels[200000:])

0.79479999999999995

In [107]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs')
clf = clf.fit(train_data[:200000], train_labels[:200000])
clf.score(train_data[200000:],train_labels[200000:])

0.73792000000000002

In [16]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_data[:200000], train_labels[:200000])
clf.score(train_data[200000:],train_labels[200000:])

0.74794000000000005

In [ ]:
from sklearn.svm import SVC
clf = SVC(probability=True)
clf.fit(train_data[:200000], train_labels[:200000])
clf.score(train_data[200000:], train_labels[200000:])

In [ ]:
#experimenting on different parameters of svm
from sklearn import svm
X,Y = train_data,train_labels
W = training_data[1:,31]
W = W.astype(np.float64)

X1 = X[:100000];
W1 = W[:100000];
Y1 = Y[:100000];
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
kernels = ['linear', 'rbf', 'poly']

print("\nExperimenting for different kernels")
for kernel in kernels:
    svc= svm.SVC(kernel=kernel).fit(X1,Y1)
#     plotSVC('kernel=' + str(kernel))
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
    print("Kernel ", kernel," Svm score for test data:",svmScore)
    
print("\nExperimenting for different gammas")
for gamma in gammas:
    svc = svm.SVC(kernel='rbf', gamma=gamma).fit(X, Y)
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
#     plotSVC(‘gamma=’ + str(gamma))
    print("Gamma:", gamma," Svm score for test data:",svmScore)
    
print("\nExperimenting for different Regularization parameters")
for c in Cs:
    svc = svm.SVC(kernel='rbf', C=c).fit(X, Y)
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
#     plotSVC(‘C=’ + str(c))
    print("C:", c," Svm score for test data:",svmScore)

print("\nExperimenting for different Degrees for polynomial kernel")
degrees = [0, 1, 2, 3, 4]
for degree in degrees:
    svc = svm.SVC(kernel='poly', degree=degree).fit(X, Y)
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
    print("Degree ", degree," Svm score for test data:",svmScore)
    


Experimenting for different Regularization parameters
('C:', 0.001, ' Svm score for test data:', 0.7679)
('C:', 0.01, ' Svm score for test data:', 0.81021)
('C:', 0.1, ' Svm score for test data:', 0.82826)
('C:', 1, ' Svm score for test data:', 0.83924)
('C:', 10, ' Svm score for test data:', 0.85133)

Experimenting for different Degrees for polynomial kernel
('Degree ', 0, ' Svm score for test data:', 0.65666)
('Degree ', 1, ' Svm score for test data:', 0.72278)
('Degree ', 2, ' Svm score for test data:', 0.80722)
('Degree ', 3, ' Svm score for test data:', 0.80736)


In [80]:
data = [[0,1],[2,1],[76,2],[311,3],[4,400]]
pca = PCA()
pca.fit(data)
print(pca.components_)

[[-0.39622191  0.91815478]
 [ 0.91815478  0.39622191]]
